In [ ]:
import time
import numpy as np

from env.cartpole import CartPole
from env.system import ControlSystem, Controller, LinearQuadraticRegulator
from base.construct import Construct
from base.utilities import *

import torch
torch.set_printoptions(precision=6)
torch.set_flush_denormal(True)
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, Dataset, random_split, TensorDataset

import matplotlib.pyplot as plt

In [ ]:
# Define the device (cpu or cuda) and the control system object.
# Send the control system object to device.

## Data generation and preparation

The cell below generates and prepares the data to be sent to the training routine.  
There is no need to change this cell. However, if you want, you can play with 
the batch_size and the total number of data points to train on.

In [ ]:
torch.manual_seed(17)

batch_size = 4
total_size = batch_size * 60

x_train_tensor = torch.empty((0,cs.robot.num_states))
x_val_tensor = torch.empty((0,cs.robot.num_states))
for _ in range(total_size):
    x_train_tensor = torch.cat((x_train_tensor,
                    torch.reshape(cs.robot.random_state(), 
                                  (1,cs.robot.num_states))), dim=0)
    x_val_tensor = torch.cat((x_val_tensor,
                    torch.reshape(cs.robot.random_state(), 
                                  (1,cs.robot.num_states))), dim=0)
y_train_tensor = torch.zeros((len(x_train_tensor), 1))
y_val_tensor = torch.zeros((len(x_val_tensor), 1))

# Build data set containing all data zpoints
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
val_dataset = TensorDataset(x_val_tensor, y_val_tensor)

# Build a loader of each set
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size)

## Create the loss function and the optimizer

The cell below defines the loss function and the optimizer.

In [ ]:
loss_fn = ... # define an L1 loss function.
optimizer = optim.Adam(cs.parameters(), lr=)  # set the learning rate (gradient descent step size) here.

In [ ]:
print(f"Number of trainable parameters: {count_parameters(cs)}")
for name, layer in cs.named_modules():
    print(name)
cs

## Test the performance of the initial neural net vs linear quadratic regulator

In [ ]:
# Define the linear quadratic regulator and print its gains.  
# Simulate 30 trajectories starting from the same initial condition (selected randomly)
# with the neural net controller and the linear quadratic regulator.
# Compute the performance of each controller and print the average performance ratio
# defined as neural net performance / lqr performance.

c = # define lqr
print(f"LQR gains: {c.K}")
perf_ratio = 0
for k in range(30):
    s0 = ...
    traj_learned, ctrl_learned = cs.robot.simulate(...)
    traj_lqr, ctrl_lqr = cs.robot.simulate(...)
    cur_ratio = cs.loss_fn(...).item()/cs.loss_fn(...).item()
    perf_ratio = perf_ratio + ...
print("--- Before training ---")
print(f"Performance ratio: {perf_ratio*100}%")

## Create the machine learning pipeline and train for n epochs.

In [ ]:
# Construct the machine learning pipeline object here.
# - It will want the control system object, the loss function, the optimizer for training.
# Send this object to the device you want the training done.
# Set training and validation data loaders for the pipeline.

prob = Construct(...)
prob.to(device)
prob.set_loaders(...)

In [ ]:
# Set the number of epochs for training and train for that many epochs.

n_epochs = ...
start_time = time.time()
prob.capture_gradients(['controller.fcl.0', 'controller.fcl.1', 'controller.fcl.2', 
                        'controller.fcl.3'])
prob.train(...)
prob.remove_hooks()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
for p in cs.parameters():
    print(p)

In [ ]:
# This cell acquires the gradients during training for observation.
# Choose which gradient you want to observe here for your report.
# Discuss the behavior of the gradients with respect to the convergence of your model.

gradients0 = np.array(prob._gradients['controller.fcl.0']['weight']).squeeze()
print(gradients0.shape)
gradients = gradients0[:,3,0]

In [ ]:
# This cell normalizes and smooths out the gradients captured during training.

corrected_gradients = calc_corrected_ewma(gradients, 19)
corrected_sq_gradients = calc_corrected_ewma(np.power(gradients, 2), 1999)
adapted_gradients = corrected_gradients / (np.sqrt(corrected_sq_gradients) + 1e-8)

In [ ]:
# This cell plots the gradients that are selected from the captured gradients.

fig, axs = plt.subplots(1, 1, figsize=(15, 5))
axs.plot(adapted_gradients[:-300], c='g', label='Adapted Gradients')
axs.set_title('Gradients')
axs.set_ylabel('Gradient')
axs.set_xlabel('Mini-batches')
axs.set_ylim([-1.5, 1.5])
axs.legend(fontsize=12)
fig.tight_layout()

## Verify that the training yields a good controller

In [ ]:
# Reperform the performance comparison between the neural net controller and LQR.
# Simulate 30 trajectories starting from the same initial condition (selected randomly)
# with the neural net controller and the linear quadratic regulator.
# Compute the performance of each controller and print the average performance ratio
# defined as neural net performance / lqr performance.

perf_ratio = 0
for k in range(30):
    s0 = ...
    traj_learned, ctrl_learned = cs.robot.simulate(...)
    traj_lqr, ctrl_lqr = cs.robot.simulate(...)
    cur_ratio = cs.loss_fn(...).item()/cs.loss_fn(...).item()
    perf_ratio = perf_ratio + ...
print("--- After training ---")    
print(f"Performance ratio: {perf_ratio*100}%")

In [ ]:
# Simulate a couple of sample trajectories starting from a random initial state.
# One of these trajectories will be controller by the neural net, the other by LQR.
# Plot the time evolution of these trajectories (x and theta) w.r.t. time on the same plots for easy comparison.
# Make the lqr trajectory dashdotted and the neural net solid.
# Also provide a plot of the used control authority, both from the lqr and the neural net.

s0 = ...
traj_learned, ctrl_learned = cs.robot.simulate(...)
traj_lqr, ctrl_lqr = cs.robot.simulate(...)
fig, axs = plt.subplots(1,2, figsize=(12,8))
axs[0].plot(...)
axs[0].plot(...)
axs[1].plot(...)
axs[1].plot(...)
fig, axs = plt.subplots(1,1, figsize=(12,8))
axs.plot(...)
axs.plot(...)

In [ ]:
# This plots training losses obtained during training and then the validation 
# losses on the same plot. 
# Interpret these plots in your report.

prob.plot_losses()